In [5]:
import pandas as pd
import numpy as np
from find_path import find_file
from operator import itemgetter
from collections import OrderedDict
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
import re
import sqlalchemy
import db_utils
import warnings
warnings.filterwarnings('ignore')

In [6]:
# # load DDD mutations
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
# query = '''select * from milena_db.DDD_mutations_mapped_filtered as df'''
# ddd = pd.read_sql_query(query, con=eng)
# ddd['chromosome'] = [int(i) if str(i).isdigit() else str(i) for i in ddd['chromosome']]

# I added one extra column to the DDD study dataset ('pathogenicity_conflict'), which I need here to add it to the merged dataset, but I didn't push this to mysql, so I will alternatively load it here:
# comes from ddd_study notebook
%store -r mapped_grouped
ddd = mapped_grouped.copy(deep=True)

In [3]:
# # One can skip all these steps and just load the already processed dataset:
# clinvar_m = pd.read_csv('output/clinvar_filtered_kristina_milena.csv', header=0, sep='\t')
# # And go to the merging DDD and Clinvar part.

# # load unfiltered clinvar
# clinvar = pd.read_csv(find_file('kristina_variant_summary.txt'), sep='\t', header=0)
# # clean it up a little
# clinvar = clinvar[clinvar['Assembly'] == 'GRCh38']
# clinvar = clinvar[['#AlleleID', 'VariationID', 'GeneSymbol', 'Chromosome', 'Start', 'ReferenceAlleleVCF', 'AlternateAlleleVCF']]
# clinvar.drop_duplicates(inplace=True)
# clinvar.reset_index(inplace=True, drop=True)

# # load filtered clinvar:
# query = '''select * from kristina_db.ClinVar_filtered as df'''
# clinvar_filtered = pd.read_sql_query(query, con=eng)

# # merge them:
# clinvar['Chromosome'] = [int(i) if str(i).isdigit() else str(i) for i in clinvar['Chromosome']]
# clinvar_filtered['Chromosome'] = [int(i) if str(i).isdigit() else str(i) for i in clinvar_filtered['Chromosome']] #I am doing this to have successful merging
# clinvar_m = clinvar_filtered.merge(clinvar, how='inner', left_on=['AlleleID','VariationID','Chromosome'], right_on=['#AlleleID','VariationID','Chromosome']) 
# clinvar_m = clinvar_m[['AlleleID', 'VariationID', 'Chromosome', 'Start', 'ReferenceAlleleVCF','AlternateAlleleVCF', 'GeneSymbol','uni_AC', 'Uniprot_iso', 'aa_change',  'ClinicalSignificance',  'PhenotypeList', 'NDD_phenotype']]

# ### clean up the merged dataframe:

# # choose and rename columns
# clinvar_m.rename(columns={'AlleleID': 'allele_id', 'VariationID':'variation_id','Chromosome':'chromosome', 'Start':'coords','ReferenceAlleleVCF':'ref','AlternateAlleleVCF':'alt','GeneSymbol':'gene_symbol','uni_AC':'uniprot_id','Uniprot_iso':'uniprot_iso_id', 'ClinicalSignificance':'pathogenicity','PhenotypeList':'phenotype','NDD_phenotype':'ndd_phe'}, inplace=True)
# # introduce the composite_id column:
# clinvar_m['composite_id'] = ['_'.join([str(i) for i in [a, b, c, d]]) for a, b, c, d in zip(clinvar_m['chromosome'], clinvar_m['coords'], clinvar_m['ref'], clinvar_m['alt'])]
# # clean up pathogenicity and phenotype columns:
# dictionary = {'Uncertain significance':'uncertain', 'Pathogenic':'pathogenic', 'Conflicting interpretations of pathogenicity':'conflicting', 'Likely pathogenic':'pathogenic', 'Benign':'benign', 'Likely benign':'benign', 'Pathogenic/Likely pathogenic':'pathogenic', 'Benign/Likely benign':'benign'}
# for k, v in dictionary.items():
#     clinvar_m['pathogenicity'] = clinvar_m['pathogenicity'].str.replace(k, v)
# dictionary = {'pathogenic/pathogenic':'pathogenic', 'benign/benign':'benign'}
# for k, v in dictionary.items():
#     clinvar_m['pathogenicity'] = clinvar_m['pathogenicity'].str.replace(k, v)
# clinvar_m['phenotype']  =[i.lower().replace('|', ' | ').replace('-', '').replace(' | not provided', '').replace(' | not specified', '').replace('not provided | ', '').replace('not specified | ', '').replace('not provided, ', '').replace(', not provided', '') for i in clinvar_m['phenotype']]
# clinvar_m['phenotype'] = clinvar_m['phenotype'].replace({'not provided':None}) #change to proper None values
# clinvar_m['phenotype'] = clinvar_m['phenotype'].replace({'not specified':None}) #change to proper None values
# # filter out Ter mutations: 
# clinvar_m = clinvar_m[~(clinvar_m['aa_change'].str.contains('Ter'))]
# # reorder the columns:
# clinvar_m = clinvar_m[clinvar_m.columns.tolist()[:6] + ['composite_id'] + clinvar_m.columns.tolist()[6:-1]]
# clinvar_m.drop_duplicates(inplace=True)
# clinvar_m.reset_index(inplace=True, drop=True)
# clinvar_m

,allele_id,variation_id,chromosome,coords,ref,alt,composite_id,gene_symbol,uniprot_id,uniprot_iso_id,aa_change,pathogenicity,phenotype,ndd_phe
0,15043,4,15,84799209,G,A,15_84799209_G_A,ZNF592,Q92610,Q92610,Gly1046Arg,uncertain,galloway mowat syndrome 1,1
1,15045,6,11,126277517,A,G,11_126277517_A_G,FOXRED1,Q96CU9,Q96CU9-1,Asn430Ser,pathogenic,"mitochondrial complex 1 deficiency, nuclear ty...",0
2,15046,214885,14,31562125,G,A,14_31562125_G_A,NUBPL,Q8TB37,Q8TB37-1,Gly56Arg,uncertain,inborn genetic diseases | mitochondrial comple...,0
3,15050,11,6,26090957,A,T,6_26090957_A_T,HFE,Q30201,Q30201-1,Ser65Cys,conflicting,hemochromatosis type 1 | hereditary hemochroma...,0
4,15051,12,6,26091078,T,C,6_26091078_T_C,HFE,Q30201,Q30201-1,Ile105Thr,uncertain,hemochromatosis type 1 | hereditary hemochroma...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329555,1330396,1339345,22,20988856,T,C,22_20988856_T_C,LZTR1,Q8N653,Q8N653,Tyr193His,pathogenic,inborn genetic diseases,0
329556,1330410,1339351,19,1469921,C,T,19_1469921_C_T,APC2,O95996,O95996-1,Pro2207Leu,pathogenic,sotos syndrome 3,1
329557,1330411,1339352,19,1457099,G,A,19_1457099_G_A,APC2,O95996,O95996-1,Val355Ile,pathogenic,sotos syndrome 3,1
329558,1330414,1339355,1,16992363,G,A,1_16992363_G_A,ATP13A2,Q9NQ11,Q9NQ11-1,Pro629Ser,uncertain,spastic tetraparesis,0


In [4]:
# #check for NaN values:
# col_list = clinvar_m.columns.tolist()
# for col in col_list:
#     if ((clinvar_m[col].isna()).any()).any():
#         print(col) 
# # phenotype contains NaN values but that's okay

phenotype


In [5]:
# # check redundancy in clinvar_m:
# clinvar_m1 = clinvar_m.copy(deep=True)
# clinvar_m1 = clinvar_m1.groupby(['chromosome','coords','ref','alt', 'composite_id'], as_index=False).agg({i: lambda x: x.tolist() for i in clinvar_m1.columns.tolist()[0:2] + clinvar_m1.columns.tolist()[7:]})

# #those that were single values, I don't want them in a list
# list_columns = clinvar_m1.columns.tolist()[5:] 
# for column in list_columns:
#     clinvar_m1[column] = [list(set(i)) if len(set(i)) > 1 else ''.join([str(a) for a in i]) for i in clinvar_m1[column].values]
    
# clinvar_m1['allele_id'] = clinvar_m1['allele_id'].astype(int)
# clinvar_m1['variation_id'] = clinvar_m1['variation_id'].astype(int)
# clinvar_m1['ndd_phe'] = clinvar_m1['ndd_phe'].astype(int)

# clinvar_m1[clinvar_m1['ndd_phe'].apply(lambda x: type(x) == list)] #none of the columns have redundancy! that's good - that means there are no cases where one gene is coding for more than one uniprot or where one mutation is falling into 2 genes and so on, that's also fine but it complicates the situation a little

# # check redundancy, but for allele_id, variation_id and chromosome:
# clinvar_m2 = clinvar_m.copy(deep=True)
# clinvar_m2 = clinvar_m2.groupby(['allele_id','variation_id','chromosome'], as_index=False).agg({i: lambda x: x.tolist() for i in clinvar_m2.columns.tolist()[3:]})

# list_columns = clinvar_m2.columns.tolist()[3:] 
# for column in list_columns:
#     clinvar_m2[column] = [list(set(i)) if len(set(i)) > 1 else ''.join([str(a) for a in i]) for i in clinvar_m2[column].values]
    
# clinvar_m2['ndd_phe'] = clinvar_m2['ndd_phe'].astype(int) 
# clinvar_m2[clinvar_m2['aa_change'].apply(lambda x: type(x) == list)] #also no redundancy, good!


,allele_id,variation_id,chromosome,coords,ref,alt,composite_id,gene_symbol,uniprot_id,uniprot_iso_id,aa_change,pathogenicity,phenotype,ndd_phe


In [6]:
# # Now, we did compare reference amino acid to swissprot sequences but we did not compare to John's version, so I will do that:

# #I need to prepare clinvar_m dataset first by introducing some columns
# clinvar_m.reset_index(inplace=True, drop=True)
# clinvar_m['aa_ref'] = [i[:3] for i in clinvar_m['aa_change']]
# clinvar_m['aa_alt'] = [i[-3:] for i in clinvar_m['aa_change']]
# clinvar_m['aa_pos'] = [int(''.join([a for a in i if a.isdigit()])) for i in clinvar_m['aa_change']]
# clinvar_m['aa_ref_short'] = clinvar_m['aa_ref'] 
# clinvar_m['aa_alt_short'] = clinvar_m['aa_alt']
# AA_dict = {'Phe':'F','Leu':'L','Ser':'S','Tyr':'Y','Cys':'C','Trp':'W','Pro':'P','His':'H','Gln':'Q','Arg':'R','Ile':'I','Met':'M','Thr':'T','Asn':'N','Lys':'K','Val':'V','Ala':'A','Asp':'D','Glu':'E','Gly':'G','Ter':'*'}
# clinvar_m['aa_ref_short'] = [AA_dict.get(i) for i in clinvar_m['aa_ref_short']]
# clinvar_m['aa_alt_short'] = [AA_dict.get(i) for i in clinvar_m['aa_alt_short']]

# # loading John's dataset
# query = '''select * from chopyan_db.uniprot_id_sequence as df'''
# john_seq = pd.read_sql_query(query, con=eng)

# compare_ref_aa = clinvar_m.merge(john_seq, how='inner', on='uniprot_id')
# compare_ref_aa = compare_ref_aa[['composite_id','uniprot_id','aa_ref_short','aa_ref','aa_pos','aa_alt_short','aa_alt', 'aa_change','sequence']]
# compare_ref_aa['length'] = [len(i) for i in compare_ref_aa['sequence'].values]

# print(len(compare_ref_aa[compare_ref_aa['aa_pos'] > compare_ref_aa['length']])) #it's either smaller or equal, good!

# #compare the actual sequences:
# compare_ref_aa['swiss_prot_ref_aa'] = [seq[pos-1] for seq, pos in zip(compare_ref_aa['sequence'].values, compare_ref_aa['aa_pos'].values)]
# print(len(compare_ref_aa[compare_ref_aa['aa_ref_short'] != compare_ref_aa['swiss_prot_ref_aa']])) #no mismatches, great!

0
0


In [7]:
# # I think I also want to run NDD annotation on this again, because I did it a bit differently from Kristina:

# ### I already made it lowercase, removed -, changed | into | with spaces around it. I also checked NaN values and there are None in this column:

# #preparing human phenotype ontology (HPO) list of NDD phenotypes and keyword search data
# #load HPO
# hpo_ndds = pd.read_excel(find_file('HPO_NDDs.xlsx'), header=0)

# #lower string, replace '-', and '|', and create a single string:
# hpo_ndds['DISEASE_NAME'] = [i.lower() for i in hpo_ndds['DISEASE_NAME']]
# hpo_ndds['DISEASE_NAME'] = hpo_ndds['DISEASE_NAME'].str.replace('-', ' ')
# hpo_string_data = ' | '.join([i for i in hpo_ndds['DISEASE_NAME']])
# filpat = r",? ?(Autosomal|Somatic) ?(Dominant|Recessive)? ?\d*|,? ?((Type)? \d\w?|Type \w{2}) ?"
# hpo = pd.DataFrame(hpo_string_data.split(' | '))
# hpo.rename(columns={0:'phenotype'}, inplace=True)
# hpo.phenotype = hpo.phenotype.str.replace(filpat, "", flags = re.IGNORECASE, regex = True)
# hpo_string_data = ' | '.join([i for i in hpo['phenotype']])

# #prepare keyword search string:
# keyword_search = ' | '.join(['neurodevelopment', 'learning disability', 'cognitive impairment', 'mental retardation', 'intellectual disability', 'autism', 'autistic', 'epilepsy', 'epileptic', 'developmental delay', 'delayed development'])
# #learning disability is a synonym of intellectual disability, but for example learning difficulties can be referring to something else, like dyslexia
# #cognitive impairment is also a synonym of intellectual disability
# #I put neurodevelopment instead of neuro because neuro will also include neuropathies and so on

# clinvar_m['hpo'] = clinvar_m['phenotype'] #had to do this because the ndd_annotation function is written for hpo and phenotype columns

# def ndd_annotation(dataset, comparison_string, string_name):
#     '''
#     Annotate phenotypes as NDD associated or not, based on a list of NDD phenotypes
#     dataset -> df; the dataset which contains 'phenotype' column which lists phenotypes associated with each variant, and 'hpo' column which lists hpo identifiers for those phenotypes
#     comparison_string -> str; string which contains a list of NDD phenotypes separated with a |; I used either a list of NDD phenotypes provided by the HPO ontology or the keyword search list I built myself
#     string_name -> str; name of the comparison_string, so that I can change the values in the corresponding columns
    
#     Returns a dataframe with 3 additional columns: one which lists phenotypes which were found to be NDD-associated, second which lists their corresponding HPO identifiers and the last one which says True
#     if at least one of the phenotypes in at least one of the patients were found to be NDD-associated. Also, I did the string matching both ways - if the phenotype in the phenotype column is in the
#     string and also if elements of the string are matching to one of the phenotypes in the phenotype column. Either way, the phenotypes which were found to be a match are listed in the f'phenotype_ndd_in_{string_name}'
#     column and those are the phenotypes that are in the phenotype column (they are not taken from the string).
#     '''
    
#     dataset[f'phenotype_ndd_in_{string_name}'] = False
#     dataset[f'hpo_ndd_in_{string_name}'] = False
#     dataset[f'ndd_in_{string_name}'] = False
    
#     dataset[f'phenotype_ndd_in_{string_name}'] = dataset[f'phenotype_ndd_in_{string_name}'].astype(dtype='object') 
#     dataset[f'hpo_ndd_in_{string_name}'] = dataset[f'phenotype_ndd_in_{string_name}'].astype(dtype='object')
#     dataset[f'ndd_in_{string_name}'] = dataset[f'phenotype_ndd_in_{string_name}'].astype(dtype='object')
    
#     for n, (phe, hpo) in enumerate(zip(dataset['phenotype'].values, dataset['hpo'].values)):
#         if n % 10000 == 0:
#             print('a 10 000 done')
#         if type(phe) != type(hpo): #I expect them to be same type and if this is not the case, I want to be alarmed
#             print('alarm')
#         if dataset.loc[n, 'phenotype'] == None:
#             dataset.at[n, f'phenotype_ndd_in_{string_name}'] = None
#             dataset.at[n, f'hpo_ndd_in_{string_name}'] = None #and the third column will say false!
#         comparison_list = list(set(comparison_string.split(' | '))) # hpo string may have some duplicates because it's basically a tree!
#         if type(dataset.at[n, 'phenotype']) == str:
#             phe_list = phe.split(' | ') #this is now a list of phenotypes
#             hpo_list = hpo.split(' | ') #I defined it here because I will use it in the first and the second if statement
#             if True in [True for i in phe_list if i in comparison_string]:
#                 dataset.at[n, f'ndd_in_{string_name}'] = True
#                 dataset.at[n, f'phenotype_ndd_in_{string_name}'] = ' | '.join([i for i in phe_list if i in comparison_string]) #if any of them are in the list
#                 get_indices = [phe_list.index(i) for i in phe_list if i in comparison_string]
#                 dataset.at[n, f'hpo_ndd_in_{string_name}'] = ' | '.join(list(np.array(hpo_list)[get_indices])) #get those exact HPO IDs also !
#             ######################
#             #now I want to check the other way around
#             if True in [True for i in comparison_list if i in phe]:
#                 dataset.at[n, f'ndd_in_{string_name}'] = True #again, immediately set to True, so if it's either the previous thing or this thing, I want it to be set to true
#                 if True not in [True for i in phe_list if i in comparison_string]: #if the previous statement is not true, then there is no need to concatenate the 2 strings
#                     #first let's get a list of matches from the comparison_list
#                     list_of_matches = [i for i in comparison_list if i in phe]
#                     #now let's get phenotypes which contain these matches!
#                     dataset.at[n, f'phenotype_ndd_in_{string_name}'] = ' | '.join(list(OrderedDict.fromkeys([i for i in phe_list for x in list_of_matches if x in i]))) #if one of the elements contains both of the matches, it will be mentioned twice and there is no need for that; for example 'autistic behavior, intellectual disability | skeletal disorder', the first element will appear twice, but this is unlikely #used set
#                     get_indices = list(OrderedDict.fromkeys([phe_list.index(i) for i in phe_list for x in list_of_matches if x in i])) #the reason I used ordereddict is because it preserves order!
#                     dataset.at[n, f'hpo_ndd_in_{string_name}'] = ' | '.join(list(OrderedDict.fromkeys(list(np.array(hpo_list)[get_indices])))) #get those exact HPO IDs also ! #used set
#                 if True in [True for i in phe_list if i in comparison_string]: #if this is a True statement, then I want to concatenate this and the previous value
#                     phe_value_1 = ' | '.join([i for i in phe_list if i in comparison_string])
#                     get_indices = [phe_list.index(i) for i in phe_list if i in comparison_string]
#                     hpo_value_1 = ' | '.join(list(np.array(hpo_list)[get_indices]))
#                     #value 2:
#                     list_of_matches = [i for i in comparison_list if i in phe]
#                     phe_value_2 = ' | '.join(list(OrderedDict.fromkeys([i for i in phe_list for x in list_of_matches if x in i])))
#                     get_indices = list(OrderedDict.fromkeys([phe_list.index(i) for i in phe_list for x in list_of_matches if x in i]))
#                     hpo_value_2 = ' | '.join(list(OrderedDict.fromkeys(list(np.array(hpo_list)[get_indices]))))
#                     #final value:
#                     phe_value = phe_value_1 + ' | ' + phe_value_2
#                     phe_value = ' | '.join(list(OrderedDict.fromkeys(phe_value.split(' | ')))) #I wanna take a set in that case #I'm afraid this might change the order but let's see, as long as we do the same thing with hpo, should be fine?
#                     hpo_value = hpo_value_1 + ' | ' + hpo_value_2
#                     hpo_value = ' | '.join(list(OrderedDict.fromkeys(hpo_value.split(' | '))))
#                     dataset.at[n, f'phenotype_ndd_in_{string_name}'] = phe_value
#                     dataset.at[n, f'hpo_ndd_in_{string_name}'] = hpo_value
#             if (True not in [True for i in phe_list if i in comparison_string]) & (True not in [True for i in comparison_list if i in phe]):
#                 dataset.at[n, f'phenotype_ndd_in_{string_name}'] = None #if we cannot find matches either way, then we set this to None and the columns will stay 'False'
#                 dataset.at[n, f'hpo_ndd_in_{string_name}'] = None
#         if type(dataset.at[n, 'phenotype']) == list: #this would be a list of strings
#             new_value_phe = []
#             new_value_hpo = []
#             for phe_string, hpo_string in zip(phe, hpo): #we go through one string first
#                 if phe_string == None: #if one of the values is None, which means that the phenotype for one patient is not available and then hpo will also be None
#                     pass
#                 else: #this is now if it's a string basically
#                     phe_list = phe_string.split(' | ') #I gotta define this now for every string of the list, if type(dataset.at[n, 'phenotype']) == list
#                     hpo_list = hpo_string.split(' | ')
#                     if True in [True for i in phe_list if i in comparison_string]:
#                         dataset.at[n, f'ndd_in_{string_name}'] = True
#                         new_value_phe = new_value_phe + [i for i in phe_list if i in comparison_string] #I don't want to append a list, then it will be a list of lists
#                         get_indices = [phe_list.index(i) for i in phe_list if i in comparison_string]
#                         new_value_hpo = new_value_hpo + list(np.array(hpo_list)[get_indices])
#                     ######################
#                     #now I want to check the other way around
#                     #I already defined the comparison list up there
#                     if True in [True for i in comparison_list if i in phe_string]:
#                         dataset.at[n, f'ndd_in_{string_name}'] = True #again, immediately set to True, so if it's either the previous thing or this thing, I want it to be set to true
#                         #first let's get a list of matches from the comparison_list
#                         list_of_matches = [i for i in comparison_list if i in phe_string]
#                         #now let's get phenotypes which contain these matches!
#                         new_value_phe = new_value_phe + list(OrderedDict.fromkeys([i for i in phe_list for x in list_of_matches if x in i]))
#                         get_indices = list(OrderedDict.fromkeys([phe_list.index(i) for i in phe_list for x in list_of_matches if x in i]))
#                         new_value_hpo = new_value_hpo + list(OrderedDict.fromkeys(list(np.array(hpo_list)[get_indices])))
#                     if (True not in [True for i in phe_list if i in comparison_string]) & (True not in [True for i in comparison_list if i in phe_string]):
#                         pass
#             if new_value_phe == []: #if there are no None values and also if none of the phenotypes match
#                 new_value_phe = None
#                 new_value_hpo = None
#             else:
#                 new_value_phe = ' | '.join(list(OrderedDict.fromkeys(new_value_phe))) #so far I was just concatenating strings to the new_value_hpo, but now i WANT
#                 new_value_hpo = ' | '.join(list(OrderedDict.fromkeys(new_value_hpo)))
#             dataset.at[n, f'phenotype_ndd_in_{string_name}'] = new_value_phe
#             dataset.at[n, f'hpo_ndd_in_{string_name}'] = new_value_hpo
#     return dataset

# # clinvar_m = ndd_annotation(clinvar_m, hpo_string_data, 'hpo')
# # clinvar_m = ndd_annotation(clinvar_m, keyword_search, 'keyword_search')

In [8]:
# clinvar_m['ndd_phe_m'] = [1 if (a or b) == True else 0 for a,b in zip(clinvar_m['ndd_in_hpo'], clinvar_m['ndd_in_keyword_search'])]
# print(sum(clinvar_m['ndd_phe']))
# print(sum(clinvar_m['ndd_phe_m'])) #my search finds a bit more matches, so I will keep this one

# keep the columns I think are worth saving in a file:
# clinvar_m = clinvar_m[clinvar_m.columns.tolist()[:10] + ['aa_change', 'aa_ref','aa_alt', 'aa_pos', 'aa_ref_short', 'aa_alt_short', 'pathogenicity', 'phenotype', 'phenotype_ndd_in_hpo','ndd_in_hpo','phenotype_ndd_in_keyword_search','ndd_in_keyword_search','ndd_phe_m']]
# clinvar_m.to_csv('output/clinvar_filtered_kristina_milena.csv', header=True, sep='\t', index=False)
# clinvar_m = pd.read_csv(find_file('clinvar_filtered_kristina_milena.csv'), sep='\t', header=0)

# Merging DDD and Clinvar

Columns I want to keep in the merged dataset:
- allele_id, variation_id
- chromosome, coords, ref, alt and composite_id
- gene symbol, uniprot_id, aa_change, aa_pos, pathogenicity, ndd_phe

In [15]:
# I got this from another notebook, called Clinvar_preprocessing
new_clinvar = pd.read_csv(find_file('clinvar_filtered_milena_2023.csv'), sep='\t', header=0)
clinvar_m = new_clinvar.copy(deep=True)

In [16]:
# clinvar_m = clinvar_m[['allele_id','variation_id','chromosome','coords','ref','alt', 'composite_id', 'gene_symbol', 'uniprot_id', 'aa_change', 'aa_pos', 'pathogenicity', 'ndd_phe']]
# clinvar_m.rename(columns={'ndd_phe_m':'ndd_phe'}, inplace=True)

ddd['allele_id'] = None
ddd['variation_id'] = None
ddd.rename(columns={'chr':'chromosome'}, inplace=True) # because sql does not accept chr as the column name, it means character

# I first wanted to look at the intersection:
m = ddd.merge(clinvar_m, how='inner', on='composite_id') #maybe I could have also merged on aa_change, but I checked and it makes no difference
m[m['gene_symbol_x'] != m['gene_symbol_y']] #there are small differences for fused genes, but I would take gene_symbol x from DDD study because it's simplified
m[m['aa_change_x'] != m['aa_change_y']] #varmap mapping I used for DDD study is clearly the same as mapping in clinvar !! great !!

## there was a conflict for pathogenicity values for 332 rows:
combinations = []
for a, b in zip(m[m['pathogenicity_x'] != m['pathogenicity_y']]['pathogenicity_x'], m[m['pathogenicity_x'] != m['pathogenicity_y']]['pathogenicity_y']):
    combinations.append([a, b])
    
print(set(tuple(row) for row in combinations)) #these are all the combinations I saw

# I want to label this flag in the 'pathogenicity_conflict' column:
dataset_conflict_ind = m[(m['pathogenicity_x'] != m['pathogenicity_y']) & (m['pathogenicity_conflict'].isna())][['pathogenicity_x','pathogenicity_y','pathogenicity_conflict']].index
both_conflict_ind = m[(m['pathogenicity_x'] != m['pathogenicity_y']) & (m['pathogenicity_conflict'] == 'patient_conflicting')][['pathogenicity_x','pathogenicity_y','pathogenicity_conflict']].index
m.loc[dataset_conflict_ind, 'pathogenicity_conflict'] = 'dataset_conflicting'
m.loc[both_conflict_ind, 'pathogenicity_conflict'] = 'both_conflicting'

#solution:
#when it's pathogenic or benign in ddd for example and uncertain in clinvar, I will label it as pathogenic or benign, respectively
#when it's the same situation, but with conflicting, I will also label it as pathogenic or benign, because probably the conflict has been resolved, or otherwise if they are not a 100% sure that it's pathogenic or benign, they should label it as uncertain or conflicting
#when it's pathogenic in one dataset and benign in another, I will label it as conflicting
#when it's uncertain and conflictig, I will label it as conflicting because that is more information, that means in one of the datasets, there is information for both

for a, b, ind in (zip(m[m['pathogenicity_x'] != m['pathogenicity_y']]['pathogenicity_x'].values, m[m['pathogenicity_x'] != m['pathogenicity_y']]['pathogenicity_y'].values, m[m['pathogenicity_x'] != m['pathogenicity_y']].index)):
    if (((a == 'pathogenic') or (a == 'benign')) and (b == 'uncertain')) or (((b == 'pathogenic') or (b == 'benign')) and (a == 'uncertain')):
        new_value = ''.join([i for i in [a, b] if i != 'uncertain'])
        m.loc[ind, 'pathogenicity_x'] = new_value
        m.loc[ind, 'pathogenicity_y'] = new_value
    if (((a == 'pathogenic') or (a == 'benign')) and (b == 'conflicting')) or (((b == 'pathogenic') or (b == 'benign')) and (a == 'conflicting')):
        new_value = ''.join([i for i in [a, b] if i != 'conflicting'])
        m.loc[ind, 'pathogenicity_x'] = new_value
        m.loc[ind, 'pathogenicity_y'] = new_value
    if ((a == 'pathogenic') and (b == 'benign')) or ((b == 'pathogenic') and (a == 'benign')):
        new_value = 'conflicting'
        m.loc[ind, 'pathogenicity_x'] = new_value
        m.loc[ind, 'pathogenicity_y'] = new_value
    if ((a == 'uncertain') and (b == 'conflicting')) or ((b == 'uncertain') and (a == 'conflicting')):
        new_value = 'conflicting'
        m.loc[ind, 'pathogenicity_x'] = new_value
        m.loc[ind, 'pathogenicity_y'] = new_value

# now also the ndd_phe column was not always a match, in other words the phenotype provided in clinvar ndd_phe is 0 and in ddd is 1 or the other way around
# I looked at those examples, and it's mostly like this when phenotype is 'None' in one of the 2 datasets, so in one of them it might be 1 and in another it might be 0 simply because the phenotype is not provided
# I looked for cases where the phenotype is provided in both datasets, and yet, in one of them it might be 0 and in another it might be 1 and this is because either the phenotypes are slightly differently stated: for example for this mutation: 10_102689939_T_C
# or the phenotypes provided were quite different
m[(m['ndd_phe_x'] == 1) & (m['ndd_phe_y'] == 0)]['composite_id'].unique().tolist()
m[(m['ndd_phe_y'] == 1) & (m['ndd_phe_x'] == 0)]['composite_id'].unique().tolist()
# clinvar_m[(clinvar_m['composite_id'].isin(m[(m['ndd_phe_x'] == 1) & (m['ndd_phe_y'] == 0)]['composite_id'].unique().tolist())) & ~(clinvar_m['phenotype'].isna())]; #those which are 0 in clinvar and ddd in 1
# ddd[(ddd['composite_id'].isin(m[(m['ndd_phe_y'] == 1) & (m['ndd_phe_x'] == 0)]['composite_id'].unique().tolist())) & ~(ddd['phenotype'].isna())];

#so I will use or:
m['ndd_phe'] = [a or b for a, b in zip(m['ndd_phe_x'], m['ndd_phe_y'])]

#choose and rename columns:
m = m[['allele_id_y','variation_id_y', 'chromosome_x', 'coords_x', 'ref_x', 'alt_x', 'composite_id', 'gene_symbol_x', 'uniprot_id_x', 'aa_change_x', 'aa_pos_x', 'pathogenicity_x', 'pathogenicity_conflict', 'ndd_phe']]
m.rename(columns={'aa_pos_x':'aa_pos', 'allele_id_y':'allele_id','chromosome_x':'chromosome', 'variation_id_y':'variation_id','coords_x':'coords','ref_x':'ref','alt_x':'alt','gene_symbol_x':'gene_symbol','uniprot_id_x':'uniprot_id','aa_change_x':'aa_change','pathogenicity_x':'pathogenicity'}, inplace=True)
clinvar_m['pathogenicity_conflict'] = None
                  
# now the clinvar_m and ddd which do not overlap (parts outside of the intersection, and in the end I will concatenate three datasets)
col_list = m.columns.tolist()
ddd = ddd[col_list]
clinvar_m = clinvar_m[col_list]

ddd = ddd[~(ddd['composite_id'].isin(m['composite_id'].unique().tolist()))]
clinvar_m = clinvar_m[~(clinvar_m['composite_id'].isin(m['composite_id'].unique().tolist()))]
ddd['ddd'] = 1
ddd['clinvar'] = 0
clinvar_m['ddd'] = 0
clinvar_m['clinvar'] = 1
m['ddd'] = 1
m['clinvar'] = 1

mutation_data = pd.concat([clinvar_m, ddd, m], axis=0)

mutation_data['ndd_phe'] = mutation_data['ndd_phe'].astype('int')

{('benign', 'uncertain'), ('uncertain', 'pathogenic'), ('pathogenic', 'conflicting'), ('benign', 'conflicting'), ('pathogenic', 'benign'), ('uncertain', 'conflicting'), ('pathogenic', 'uncertain'), ('uncertain', 'benign'), ('benign', 'pathogenic')}


## Investigating NaN values and redundancy

In [35]:
max([len(str(i)) for i in mutation_data['gene_symbol']]) # here I noticed that within clinvar there are values like: covers 10 genes, none of which show dosage sensitivity, which I don't really know what it means and since uniprot was provided, I just replaced this with a proper gene symbol
mutation_data.loc[mutation_data['uniprot_id'] == 'P22309', 'gene_symbol'] = 'UGT1A1' # replaced this with manually searched gene symbol
#however, in clinvar there are still values where genes are separated by ';' and one of them is really affected by the mutation according to the locations, but another one is not necessarily affected, so be aware of that - there are 712 cases like this
#for now I didn't do anything about this because for overlapping with John's predictor, we actually need uniprots and not gene symbols!

#check for NaN values:
col_list = mutation_data.columns.tolist()
for col in col_list:
    if ((mutation_data[col].isna()).any()).any():
        print(col) 
#none of the columns contain NaN values

#uniprot was important for us not to have this, because we used that for the overlap !

allele_id
variation_id
pathogenicity_conflict


In [ ]:
# First I thought there is redundancy in DDD because one mutation can be found in 2 genes and 2 uniprots and so on, but if you use composite_id + aa_change, there is no redundancy
# In clinvar there is this redundancy because the identifier does not include anything on the protein level, but it's kept in one row, separated by ; - in 712 rows

# So to answer the question if one mutation is one row - for clinvar yes because other information is concatenated.
# In ddd if you just use composite ID (look at the mutation from the gene level), then no, but if you include aa_change, then yes!

In [44]:
# Does one clinvar ID correspond to one DDD id? Yes.

# for those cases where allele id and variation id is not NaN, I wanted to check if one composite ID-aa change combination corresponds to one allele and var. ID and chromosome
mut_data = mutation_data[~(mutation_data['allele_id'].isna()) & ~(mutation_data['variation_id'].isna())]

mut_data1 = mut_data.copy(deep=True)
mut_data1 = mut_data1[['composite_id', 'aa_change'] + [i for i in mut_data1.columns.tolist() if (i != 'composite_id') and (i != 'aa_change')]]
mut_data1 = mut_data1.groupby(['composite_id','aa_change'], as_index=False).agg({i: lambda x: x.tolist() for i in mut_data1.columns.tolist()[2:]})

list_columns = mut_data1.columns.tolist()[2:] 
for column in list_columns:
    mut_data1[column] = [list(set(i)) if len(set(i)) > 1 else ''.join([str(a) for a in i]) for i in mut_data1[column].values]

mut_data1[mut_data1['allele_id'].apply(lambda x: type(x) == list)] # there is no redundancy

# and to check the other way around too:
mut_data1 = mut_data.copy(deep=True)
mut_data1 = mut_data1[['allele_id', 'variation_id', 'chromosome'] + [i for i in mut_data1.columns.tolist() if (i != 'allele_id') and (i != 'variation_id') and (i != 'chromosome')]]
mut_data1 = mut_data1.groupby(['allele_id','variation_id', 'chromosome'], as_index=False).agg({i: lambda x: x.tolist() for i in mut_data1.columns.tolist()[3:]})

list_columns = mut_data1.columns.tolist()[3:] 
for column in list_columns:
    mut_data1[column] = [list(set(i)) if len(set(i)) > 1 else ''.join([str(a) for a in i]) for i in mut_data1[column].values]

mut_data1[mut_data1['composite_id'].apply(lambda x: type(x) == list)] # there is no redundancy

# a simpler way to do this would be but I wanted to check both ways, just to be on the safe side
mut_data = mut_data[~(mut_data['allele_id'].isna()) & ~(mut_data['variation_id'].isna())]
len(mut_data[['allele_id','variation_id','chromosome']].drop_duplicates()) == len(mut_data[['composite_id','aa_change']].drop_duplicates())

KeyboardInterrupt: 

## Upload to MySQL

In [47]:
clinvar_v1 = pd.read_csv(find_file('clinvar_filtered_kristina_milena.csv'), sep='\t', header=0)
mutation_data['clinvar_version'] = None
mutation_data.loc[mutation_data['composite_id'].isin(clinvar_v1['composite_id'].tolist()), 'clinvar_version'] = 'v1'
mutation_data.loc[~mutation_data['composite_id'].isin(clinvar_v1['composite_id'].tolist()), 'clinvar_version'] = 'v2'
mutation_data.loc[mutation_data['clinvar'] == 0, 'clinvar_version'] = None

In [13]:
#  CREATE TABLE `DDD_and_clinvar_mutations_2023_release` (
#    `allele_id` int(8) DEFAULT NULL,
#    `variation_id` int(8) DEFAULT NULL,
#    `chromosome` varchar(2) NOT NULL,
#    `coords` int(10) NOT NULL,
#    `ref` text(1) NOT NULL,
#    `alt` text(1) NOT NULL,
#    `composite_id` varchar(18) NOT NULL,
#    `gene_symbol` varchar(67) NOT NULL,
#    `uniprot_id` varchar(11) NOT NULL,
#    `aa_change` varchar(12) NOT NULL,
#    `aa_pos` int(6) NOT NULL,
#    `pathogenicity` text(12) NOT NULL,
#    `pathogenicity_conflict` text(21) DEFAULT NULL,
#    `ndd_phe` int(1) NOT NULL,
#    `ddd` int(1) NOT NULL,
#    `clinvar` int(1) NOT NULL,
#    `clinvar_version` varchar(3) DEFAULT NULL,
#    PRIMARY KEY (`composite_id`,`aa_change`)
#  ) ENGINE=MyISAM DEFAULT CHARSET=utf8mb4;

In [87]:
# mutation_data.reset_index(inplace=True, drop=True) 
# # make sure NaN values are None

connect = db_utils.get_connection()
cursor = connect.cursor()

db_table = 'milena_db.DDD_and_clinvar_mutations_2023_release'
query = """insert into %s 
        (allele_id, variation_id, chromosome, coords, ref, alt, composite_id, gene_symbol, uniprot_id, aa_change, aa_pos, pathogenicity, pathogenicity_conflict, ndd_phe, ddd, clinvar, clinvar_version)
        values (%%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s)""" % (db_table)

for i in range(mutation_data.shape[0]):
    cursor.execute(query, tuple(mutation_data.loc[i]))

## Plotting

In [15]:
# number of DDD and clinvar mutations

out = venn2(subsets=(20000, 328149, 1411), set_labels=['DDD', 'ClinVar'])

for idx, subset in enumerate(out.subset_labels):
    out.subset_labels[idx].set_visible(False)
    
out.get_patch_by_id('01').set_color('#3D3D3D')
out.get_patch_by_id('01').set_edgecolor('black')

out.get_patch_by_id('11').set_color('black')
out.get_patch_by_id('11').set_edgecolor('black')
out.get_patch_by_id('11').set_alpha(0.7)

out.get_patch_by_id('10').set_color('#11FF00')
out.get_patch_by_id('10').set_edgecolor('black')

plt.savefig('output/plots/venn_ddd_clinvar_intersection1.jpeg')

In [16]:
# plotting the proportion of pathogenic mutations

out = venn2(subsets=(0.3129, 0.09825, 0.55492), set_labels=['DDD', 'ClinVar'])

for idx, subset in enumerate(out.subset_labels):
    out.subset_labels[idx].set_visible(False)
    
out.get_patch_by_id('01').set_color('#3D3D3D')
out.get_patch_by_id('01').set_edgecolor('black')

out.get_patch_by_id('11').set_color('black')
out.get_patch_by_id('11').set_edgecolor('black')
out.get_patch_by_id('11').set_alpha(0.7)

out.get_patch_by_id('10').set_color('#11FF00')
out.get_patch_by_id('10').set_edgecolor('black')

plt.savefig('output/plots/venn_ddd_clinvar_intersection2.jpeg')

In [17]:
# plotting the proportion of NDD mutations
out = venn2(subsets=(3139/(3139+941), 123876/(123876+204273), 1298/(1298+113)), set_labels=['DDD', 'ClinVar'])

for idx, subset in enumerate(out.subset_labels):
    out.subset_labels[idx].set_visible(False)
    
out.get_patch_by_id('01').set_color('#3D3D3D')
out.get_patch_by_id('01').set_edgecolor('black')

out.get_patch_by_id('11').set_color('black')
out.get_patch_by_id('11').set_edgecolor('black')
out.get_patch_by_id('11').set_alpha(0.7)

out.get_patch_by_id('10').set_color('#11FF00')
out.get_patch_by_id('10').set_edgecolor('black')

plt.savefig('output/plots/venn_ddd_clinvar_intersection3.jpeg')

In [18]:
import matplotlib.pyplot as plt

y = list(reversed(['VUS', 'benign', 'pathogenic', 'conflicting']))
 
# getting values against each value of y
x = [240487, 38933, 34303, 19917]
x.reverse()
plt.barh(y, x, color=['#FFD321','#DF8B88','#88A8DF','#727272'])
 
# setting label of x-axis
plt.xlabel("%") 
plt.title("Pathogenicity of DDD and ClinVar mutations")

plt.savefig('output/plots/clinical_interpretation.jpeg')
plt.show()